# PPA3 Database Table Viewer
If PPA is live and people are using it, and you want to view tables, please do so through this tool rather than Arc Pro. Viewing in Pro will create a schema lock on the tables, so when a user runs the tool, it will cause an error because it cannot log the run output to the locked database tables.

This viewer gets around this issue by creating an in-memory copy of desired gdb tables.

In [12]:
import pandas as pd

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

file_gdb = r'\\arcserver-svr\D\PPA3_SVR\PPA3_GIS_SVR\PPA3_run_data.gdb'

arcpy.env.workspace = file_gdb

fc_pmaster = 'project_master'
f_uid = 'project_uid'


df_pmaster = pd.DataFrame.spatial.from_featureclass(fc_pmaster)

# print([fc for fc in arcpy.ListFeatureClasses()])


## View of most recent entries into master table

In [10]:
df_pmaster.sort_values(by='time_created', ascending=False).head()

,OBJECTID,project_uid,proj_name,juris,proj_type,aadt,pci,posted_speed,len_mi,comm_type,time_created,user_email,perf_outcomes,for_review,SHAPE
217,511,8a60fda9-b751-47d5-a9f7-6c78e8f48cee,test,test,Arterial State of Good Repair,0.0,0.0,0.0,0.500076,Established Communities,2023-01-18 13:23:36,dconly@sacog.org,Promote Complete Streets and State of Good Repair,NaN,"{""paths"": [[[-13501294.3105, 4700339.189199999..."
216,510,9b13d10d-008f-4913-a13d-1e9be5897ad9,I_8-Segments_1_3_hari-v2,Yolo and Sacramento,Freeway Expansion,183339.0,70.0,65.0,14.139261,Ag,2023-01-17 16:36:29,harikishan.perugu@dot.ca.gov,Reduce VMT,NaN,"{""paths"": [[[-13553913.6923, 4654483.787500001..."
215,509,9dc6e984-f509-4336-bd72-f6f2f35ec805,I_80_segment 1_2_hari_v2,Yolo and Sacramento,Freeway Expansion,168670.0,70.0,65.0,14.287878,Established Communities,2023-01-17 16:34:54,harikishan.perugu@dot.ca.gov,Reduce VMT,NaN,"{""paths"": [[[-13554036.9417, 4654524.870700002..."
214,508,b21d150a-ba7a-4555-9379-2d07203072fb,I_80_Segments1_2_3_Hari_v2,Yolo and Sacramento,Freeway Expansion,171958.0,70.0,65.0,17.515255,Established Communities,2023-01-17 16:33:07,harikishan.perugu@dot.ca.gov,Reduce VMT,NaN,"{""paths"": [[[-13554078.0249, 4654607.037000000..."
213,507,2d70ada7-2d14-4877-b7d7-92a375127ccd,I_80_Segments1_2_3_Hari,Yolo and Sacramento,Freeway Expansion,183169.0,70.0,65.0,17.410044,Established Communities,2023-01-17 15:55:08,harikishan.perugu@dot.ca.gov,Reduce VMT,NaN,"{""paths"": [[[-13553954.7754, 4654360.538099997..."


## Find UID values that are in any of the data tables but not in the master table

In [25]:


def get_uids(in_tbl):
    uids = []

    try:
        with arcpy.da.SearchCursor(in_tbl, field_names=[f_uid]) as cur:
            for row in cur:
                uids.append(row[0])
    except:
        print(f"{in_tbl} does not have field {f_uid}. Skipping...")
        
    return uids

data_tables = [t for t in arcpy.ListTables()]
pmaster_uids = df_pmaster[f_uid].to_list()

out_dict = {}

fake_id = 'UID_NOT_FOUND' # placeholder ID used when running tool locally for testing

for t in data_tables:
    tbl_uids = get_uids(t)
    
    uids_not_in_master = []
    for uid in tbl_uids:
        if uid not in pmaster_uids and uid != fake_id:
            uids_not_in_master.append(uid)
            
    out_dict[t] = uids_not_in_master
    if len(out_dict[t]) > 0:
        print(f"table {t} has the follwing UIDs not in the project_master table: {out_dict[t]}")
        
total_missing = sum([len(v) for tbl, v in out_dict.items()])
if total_missing == 0: print("no IDs found in subreport tables that are not also in project_master")
    

TEST_TABLE does not have field project_uid. Skipping...
no IDs found in subreport tables that are not also in project_master
finished searching for project UID values that are in subreport tables but not project_master table


In [22]:
len(out_dict[t])

0